<a href="https://colab.research.google.com/github/vukovic-marko/projekat-nm/blob/master/dataset_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import gc

In [2]:
data = pd.read_json('/content/drive/My Drive/goodreads_reviews_spoiler.json.gz', lines=True, compression='gzip')

In [3]:
data.head()

,user_id,timestamp,review_sentences,rating,has_spoiler,book_id,review_id
0,8842281e1d1347389f2ab93d60773d4d,2017-08-30,"[[0, This is a special book.], [0, It started ...",5,True,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb
1,8842281e1d1347389f2ab93d60773d4d,2017-03-22,"[[0, Recommended by Don Katz.], [0, Avail for ...",3,False,16981,a5d2c3628987712d0e05c4f90798eb67
2,8842281e1d1347389f2ab93d60773d4d,2017-03-20,"[[0, A fun, fast paced science fiction thrille...",3,True,28684704,2ede853b14dc4583f96cf5d120af636f
3,8842281e1d1347389f2ab93d60773d4d,2016-11-09,"[[0, Recommended reading to understand what is...",0,False,27161156,ced5675e55cd9d38a524743f5c40996e
4,8842281e1d1347389f2ab93d60773d4d,2016-04-25,"[[0, I really enjoyed this book, and there is ...",4,True,25884323,332732725863131279a8e345b63ac33e


In [4]:
data['has_spoiler'].describe()

count     1378033
unique          2
top         False
freq      1288406
Name: has_spoiler, dtype: object

In [5]:
1378033-1288406

89627

In [6]:
89627/1378033

0.06503980673902585

In [7]:
grouped = data.groupby('has_spoiler')
grouped = grouped.apply(lambda x: x.sample(n=50000, replace=True))

In [8]:
grouped = grouped.reset_index(drop=True)

In [9]:
grouped

,user_id,timestamp,review_sentences,rating,has_spoiler,book_id,review_id
0,087f910db26d3e0bd5cee5480b051b21,2015-01-24,"[[0, Addison Coleman always knows the right ch...",5,False,11988046,8f502e9b393cb02a41ecb51a45a0f6f6
1,2c55431540a1d77217734bf959dc6291,2015-06-26,"[[0, Except for some typical YA-ness, this boo...",4,False,19695718,3df586d4f96275b7e4c6f17444d79bcd
2,bf0107e8f7219d77d86341d817231cfc,2012-08-07,"[[0, FAST read.], [0, Finished it in less than...",5,False,3636,530775191174fd251c4a88c1e7d05827
3,b68a14ad3bc75407adc8c8ffde2ac8d6,2017-04-28,"[[0, I thought I would love this book.], [0, I...",3,False,32075854,7478a839aeeab55d82fed4e501b5d4c4
4,5f3c84b201c0ff06d486977d5b5adcaf,2012-07-16,"[[0, I liked it.There where a lot of predictab...",4,False,12686642,803b719d03428b96cc130468756fb1d9
...,...,...,...,...,...,...,...
99995,3d9312557aaab21a346cb0aafe35a202,2011-12-26,"[[0, Sentence: I sentence Elizabeth Scott to a...",2,True,9742401,448db691fd81a6f1700ad2b6eeaf03df
99996,874ac23e8e42584b63a2ecff1244d5b1,2014-04-23,"[[0, It is not so unusual that a novel spannin...",4,True,6280118,f5c787005382ca447fa5f8ad248a0816
99997,ee5b02d2e644bb2e1250293f8e4275aa,2016-10-04,"[[0, I really enjoyed this story.], [1, I hate...",4,True,26156987,8e50da17b850d2f71a8db357c77ad3e8
99998,d338d9fdbf84d31baf6ea1841385461c,2015-01-07,"[[0, Okay, please be kind to me, I haven't don...",4,True,22724488,e7633eb0dca4d1dae33fb218ec2d0fbb


In [10]:
books = grouped.groupby('book_id')['review_id']
books.nunique()
# books.nunique().to_csv('/content/drive/My Drive/grouped_distribution.csv')

In [12]:
books_full = data.groupby('book_id')['review_id']
# books_full.nunique().to_csv('/content/drive/My Drive/full_distribution.csv')

In [13]:
x_train = []
y_train = []
z_train = []

for review in grouped['review_sentences']:
  x = []
  y = []
  for review_sentence in review:
    is_spoiler = review_sentence[0]
    sentence = review_sentence[1]
    x.append(sentence)
    y.append(is_spoiler)
  if 1 in y:
    z_train.append(True)
  else:
    z_train.append(False)
  
  x_train.append(' '.join(x))
  y_train.append(np.array(y))

In [14]:
df = pd.DataFrame()
df['sentences'] = x_train
df['spoilers'] = np.array(y_train)
df['is_spoiler'] = np.array(z_train)

In [15]:
df

,sentences,spoilers,is_spoiler
0,Addison Coleman always knows the right choice ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",False
1,"Except for some typical YA-ness, this book is ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",False
2,FAST read. Finished it in less than a day. It'...,"[0, 0, 0, 0, 0, 0]",False
3,I thought I would love this book. I wanted to....,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",False
4,I liked it.There where a lot of predictable th...,"[0, 0]",False
...,...,...,...
99995,Sentence: I sentence Elizabeth Scott to anothe...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, ...",True
99996,It is not so unusual that a novel spanning 20 ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, ...",True
99997,I really enjoyed this story. I hate to only gi...,"[0, 1]",True
99998,"Okay, please be kind to me, I haven't done too...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, ...",True


In [ ]:
df.to_pickle('/content/drive/My Drive/df.pkl')